In [49]:
class NeoDecisionTreeClassifier:
    def __init__(self):
        pass
    def fit(self,X,y):
        pass
    def __repr__(self):
        return "AAAA"


In [50]:
#Toy dataset
#column3 is label
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

X_train = [row[0:2] for row in training_data]
y_train = [row[2] for row in training_data]
#print(y_train)
#X_train = training_data[:]
#print(X_train)

#Column labels.
header = ["color", "diameter", "label"]

model = NeoDecisionTreeClassifier()
model
#model.fit(X_train,y_train)

AAAA

In [67]:
# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]
X_test = [row[0:2] for row in testing_data]
y_test = [row[2] for row in testing_data]



In [223]:

class treenode:
    def __init__(self,val=0,l=None,r=None, p=None):
        self.value = val
        self.leftchild = l
        self.rightchild = r  
        self.parent = p
        
class binary_tree:
    def __init__(self,val=0):
        self.root = treenode(val)

    def add(self,val,node=None):
        if (node == None):
            node = self.root
        if (node):
            if (val > node.value):
                if (node.rightchild):
                    #go next node
                    self.add(val,node.rightchild)
                else:
                    #create node
                    node.rightchild = treenode(val)
                    node.rightchild.parent = node
            else:
                if (node.leftchild):
                    #go next node
                    self.add(val,node.leftchild)
                else:
                    #create node
                    node.leftchild = treenode(val)
                    node.leftchild.parent = node

    def traversal(self,node,path):
        if (node):
            #print(node.value)
            path.extend([node])
            self.traversal(node.leftchild,path)
            self.traversal(node.rightchild,path)

    def is_leaf(self,node):
        if ((node.leftchild == None) and (node.rightchild == None)):
            return True
        return False
    def move_to_parent(self,node,count):
        if (node != self.root):
            count = self.move_to_parent(node.parent,count + 1)
            return count
        else:
            return count
    def __repr__(self):
        path = []
        msg=''
        self.traversal(self.root,path)
        msg += "traversal: %s\n"%([n.value for n in path])
        depth = 0
        for n in path:
            if self.is_leaf(n):
                cnt = self.move_to_parent(n,0)
                depth = max(depth,cnt)
        msg += "depth: %d\n"%(depth)
        return msg
    pass

t = binary_tree(50)
for i in range(1,100):
    t.add(i)

print(t)



traversal: [50, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
depth: 50



In [170]:
t.root.leftchild.rightchild.value

2

In [374]:

class Condition:
    def __init__(self,val,column):
        self.val = val
        self.column = column
        pass  
    def __repr__(self):
        if (isinstance(self.val, int) or isinstance(self.val, float)):
            msg = 'Is data[%d] >= %s' % (self.column,str(self.val))
        else:
            msg = 'Is data[%d] == %s' % (self.column,self.val)
        return msg
    def match(self, test_data):
        if (isinstance(self.val, int) or isinstance(self.val, float)):
            return (test_data[self.column] >= self.val) 
        else:
            return (test_data[self.column] == self.val)           

class decision_node:
    def __init__(self,condition,true_branch=None,false_branch=None, p=None):
        self.condition = condition
        self.true_branch = true_branch
        self.false_branch = false_branch 
        self.parent = p
        
class decision_tree:
    def __init__(self,X,y):
        transport = list(zip(*X))
        transport.append(y)
        self.data_with_label = list(zip(*transport))
        self.find_bast_condition(self.data_with_label)
        
    def find_bast_condition(self,data_with_label):
        current_uncertainty = self.gini([row[-1] for row in data_with_label])
        n_features = len(data_with_label[0])-1
        max_gain = 0
        best_condition = None
        for i in range(n_features):
            sets = set([row[i] for row in data_with_label])
            for j in sets:
                c = Condition(j,i)
                left_data,right_data = self.split_by_condition(data_with_label,c)
                gain = self.gini_gain(left_data, right_data, current_uncertainty)
                if (max_gain < gain):
                    best_condition = c
                    max_gain = gain
        print("max_gain=%f"%max_gain)
        print("%s"%best_condition)
        return max_gain, best_condition

    def gini(self, data):
        count = self.gen_count_dict(data)
        impurity = 1
        for d in count:
            impurity -= (count[d] / float(len(data)))**2
        return impurity
    def gini_gain(self, left_data, right_data, current_uncertainty):
        p = float(len(left_data)) /  float(len(left_data)+len(right_data))
        return p*self.gini(left_data) + (1-p)*self.gini(right_data)
    def gen_count_dict(self,data):
        count = {}
        for val in data:
            if val not in count:
                count[val] = 0
            count[val] += 1                 
        return count
    def split_by_condition(self,data,condition):
        #return 2 sets
        true_set = []
        false_set = []
        for val in data:
            if condition.match(val):
                true_set.extend([val])
            else:
                false_set.extend([val])
        return  true_set, false_set     
        pass
    def add_true_branch_node(self,node,condition):
        node.true_branch = decision_node(condition)
    def add_false_branch_node(self,node,condition):
        node.false_branch = decision_node(condition)
    def __repr__(self):
        return ''

training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

X_train = [row[0:2] for row in training_data]
y_train = [row[2] for row in training_data]

dt = decision_tree(X_train,y_train)
#dt.add_true_branch_node(dt.root,Condition(1))

max_gain=0.720000
Is data[1] >= 1
